#####################################################################
#########   Licence 3 ISEI - Cours de Traitement d'Images   #########
#####################################################################

Les cellules de code suivantes présentent différentes opérations 
pour importer, analyser et modifier les images via Python, 
en utilisant la librairie Scikit-Image.

Tous ces exemples sont tirés de la section Examples du site officiel 
de la librairie, qui peut être trouvée ici : 
https://scikit-image.org/docs/stable/auto_examples/index.html

Rappel, pour installer les librairies nécessaires, il faut:
- Créer un environnement d'éxécution Python local (recommandé)
- OU utiliser un environnement global installé dans votre Windows
- Utiliser la commande : pip install scikit-image scikit-image[optional] scikit-image[data] pooch pathlib numpy matplotlib plotly pathlib

In [ ]:
# Importation des librairies logicielles utilisées

# Librairies dédiées au traitement d'images
import skimage # scikit-image, qui contient toutes les fonctions que nous allons utiliser
import numpy as np # numpy, utilisée par scikit-image pour transformer les images en tableaux à plusieurs dimensions

# Librairies utilitaires
import pathlib # Pour charger automatiquement le dataset d'images
import math
import random
import matplotlib # matplotlib, pour l'affichage des images et graphiques
import matplotlib.pyplot as plt

In [ ]:
# Importation du dataset d'images
images = [f for f in pathlib.Path('CHEMIN/VERS/VOTRE/DATASET').iterdir()] # Récupération automatique des images sous la forme d'un tableau de WindowsPath

print(len(images)) # Affiche la longueur de la liste pour vérification

In [ ]:
# Déclaration de quelques fonctions utilitaires

# chooseRandomImg : pour charger une image aléatoire parmi notre dataset
def chooseRandomImg(dataset):
    index = random.randint(0, len(dataset) - 1)
    titre = images[index].name
    img = plt.imread(images[index])
    # La fonction imread de l'objet plt permet de charger une image sous la forme d'un tableau multidimensionnel, aka array Numpy
    # Une image en niveaux de gris (tableau monocouche) se présente sous la forme (hauteur, largeur)
    # Une image en couleurs (tableau à trois voir 4 couches) se présente sous la forme (hauteur, largeur, nombre de couches)

    # On peut vérifier la forme du tableau grâce à la commande : print(img.shape)

    return titre, img



# centerCrop : Fonction pour cropper des images depuis leur centre
def centerCrop(img, width=None, height=None):
    assert width or height, 'Précisez au moins une dimension' # On s'assure d'avoir au moins une dimension

    # Dans le cas où l'on aurait qu'une seule dimension transmise, on l'assigne aux deux variable spour obtenir un format carré
    width = width or height
    height = height or width

    # Récupération des dimensions de l'image
    org_height, org_width = img.shape[:2] # Permet d'assigner aux deux variables créées simultanément les valeurs des deux premières dimension du tableau (hauteur, largeur)
    cropCenter_x, cropCenter_y = org_width // 2, org_height // 2 # Calcule le point d'origine à partir duquel croper, c-à-d le centre de l'image
    dimCrop_x, dimCrop_y = width // 2, height // 2 # On calcule la distance entre le centre et la limite de la zone croppée, en divisant les dimensions demandées par 2

    # Pour cropper une image, on doit simplement délimiter la zone désirée sous la forme img[cellule Y de départ : cellule Y limite, cellule X de départ, cellule X limite]
    # Ici, on utilisera la coordonnée du centre sur l'axe y - la distance de crop en y : la coordonnée du centre sur l'axe y + la distance de crop en y,
    # la même chose sur l'axe des x
    return img[cropCenter_y-dimCrop_y : cropCenter_y + dimCrop_y, cropCenter_x-dimCrop_x : cropCenter_x + dimCrop_x]



# afficherImgHist : Fonction pour afficher une image et ses histogrammes (valeurs de pixels notées de 0 à 255)
def afficherImgHist(img, axes, bins=256, titre=None):
    # On créé les variables qui vont contenir nos différents axes
    ax_img, ax_hist = axes #ax_img va afficher l'image, ax_hist afficher l'histogramme
    ax_cumul = ax_hist.twinx() # la fonction twinx sert à jumeler deux axes sur leur abscisse, ici l'histogramme et l'histogramme cumulatif

    #Affichage de l'image dans l'ax_img
    if img.ndim < 3: # Si l'image possède moins de trois couches
        ax_img.imshow(img, cmap=plt.cm.gray) # On l'affiche en noir et blanc
    else:
        ax_img.imshow(img)
    
    ax_img.set_axis_off() # en lui enlevant ses repères
    ax_img.set_title(titre)

    #Affichage de l'histogramme, setup des axes 
    ax_hist.hist(img.ravel(), bins=bins, histtype='step', color='black') # On affiche l'histogramme avec la fonction matplotlib dédiée
    ax_hist.ticklabel_format(axis='y', style='scientific', scilimits=(0,0))
    ax_hist.set_xlabel('Valeur de pixels')
    ax_hist.set_xlim(0,255) # On note la valeur des pixels de 0 à 255 sur l'axe des X
    ax_hist.set_yticks([])

    #Affichage de l'histogramme cumulatif
    img_cumul, bins = skimage.exposure.cumulative_distribution(img, bins) # On utilise, pour l'histogramme cumulatif, la fonction de la librairie skimage.exposure
    ax_cumul.plot(bins, img_cumul, 'r') # L'histogramme cumulatif n'est pas une image mais une courbe, on utilise .plot et non pas imshow. 'r' passe la courbe en rouge
    ax_cumul.set_yticks([])

    # On retourne les trois axes
    return ax_img, ax_hist, ax_cumul

In [ ]:
# Fonction de base : Afficher une image aléatoire du dataset dans un plot
# On récupère l'image et le nom de son fichier via la fonction qu'on a écrit 
titre, img = chooseRandomImg(images)

plt.figure() # On créé une figure
plt.title(titre) # On lui donne un titre
plt.imshow(img) # On affiche l'image

In [ ]:
# Affichage de tout le dataset dans une figure sous forme de grille avec des subplots
colonnes = 3 # On veut une grille de trois colonnes
lignes = rows = math.ceil((len(images) - 1)/colonnes) # Et on calcule automatiquement l enombre de lignes nécessaires selon la taille du dataset

# On créée un objet contenant la figure globale, et un tableau à une ou deux dimensions d'objets axes
fig, axs = plt.subplots(ncols=colonnes, nrows=lignes, figsize=(12,12)) # L'argument figsize prend un couple de valeurs, et gère la taille d'affichage des subplots

# On boucle à travers les différents axes
for i, ax in enumerate(axs.flat): # L'option .flat permet de boucler à travers un tableau à deux dimensions avec un seul index
    ax.set_axis_off() # Cache les repères lors de l'affichage de l'image

    img = plt.imread(images[i])

    if img.ndim < 3: # Si l'image chargée possède moins de trois dimensions (image monocouche en noir et blanc)
        ax.imshow(img, cmap = plt.cm.gray) # On affiche l'image dans un espace colorimétrique correspondant
    else :
        ax.imshow(img)

fig.tight_layout() # Sert à réduire le plus possible les marges entre les images

In [ ]:
# Modifier les dimensions d'une image : Rescale, resize, et downscale local mean
title, img = chooseRandomImg(images)

# Rescale redimensionne une image d'après une facteur donné 
# Elle prend en arguments : l'image à rescaler, le facteur d'agrandissement (> 1) ou réduction (< 1), une option de lissage des pixels (anti_aliasing)
# Et l'option channel_axis, qu'on reverra souvent, et qui doit valoir -1 en cas de travail sur une image RGB
img_rescale = skimage.transform.rescale(img, 4, anti_aliasing=False, channel_axis=-1) # Exemple, on multiplie par 4 les dimensions de l'image (4 fois plus de pixels sur chaque dimension)

# Resize redimensionne une image aux dimensions fournies, y compris dans sa troisième dimension (nombre de couches).
# Elle prend en arguments : l'image, les dimensions sous la forme (hauteur, largeur, nombre de couches), et une option de lissage
img_resize = skimage.transform.resize(img, (250, 450, 1), anti_aliasing=True)

# Downscale_local_mean réduit une image d'après autant de facteur qu'elle a de dimensions
# Par exemple, une image RGB (notée (hauteur, largeur, 3)) devra se voir fournie trois facteurs, alors qu'une image N&B (notée (hauteur, largeur)) n'en nécessitera que deux
img_downscale = skimage.transform.downscale_local_mean(img, (4,2,3)) # Ici on réduit la troisième dimension de l'image par 3, divisant par 3 son nombre de couches, pour l'amener à 1

# On affiche les différents résultats
fig, axs = plt.subplots(nrows = 2, ncols = 2, figsize=(10,10))

# On va lister les axes les uns après les autres "à la main"
# A noter, les coordonnées des axes s'écrivent sous la forme [y,x]

# Premier axe, en haut à gauche
axs[0,0].imshow(img)
axs[0,0].set_title('image de base')

# Deuxième, en haut à droite
axs[0,1].imshow(img_rescale)
axs[0,1].set_title('Résultat rescale')

# Troisième, en bas à gauche
axs[1,0].imshow(img_resize, cmap=plt.cm.gray)
axs[1,0].set_title('Résultat resize')

# Quatrième, en bas à droite
axs[1,1].imshow(img_downscale, cmap=plt.cm.gray)
axs[1,1].set_title('Résultat downscale')

fig.tight_layout()

In [ ]:
# Cropper les images
# On va utiliser la fonction centerCrop écrite plus haut

title, img = chooseRandomImg(images) # On charge une image aléatoire

cropped_img = centerCrop(img, 150) # On la croppe au format carré en ne donnant qu'une dimension

# Et on affiche le résultat
fig, axs = plt.subplots(ncols=2, nrows=1,figsize=(20,20))

# Puisqu'on n'a qu'une seule dimension (une seule ligne ou une seule colonne) dans notre grille, les axes se notent au format [0] et non pas [0,0]
axs[0].imshow(img)
axs[0].set_title('Image de base')

axs[1].imshow(cropped_img)
axs[1].set_title('Image cropée')

fig.tight_layout()

In [ ]:
# Passer une image en niveaux de gris
titre, img = chooseRandomImg(images)

img_NB = skimage.color.rgb2gray(img) # On utilise la fonction color.rgb2gray pour transformer une image RGB en niveaux de gris
# A NOTER : il faut être sûr que l'image qu'on envoie à la fonction est bien en RGB, sous peine d'erreur
# Il existe tout un ensemble de fonctions équivalentes pour transformer l'espace colorimétrique d'une image : rgb2gray, gray2rgb, rgb2hsv, etc.

fig, axs = plt.subplots(ncols=2, nrows=1,figsize=(20,20))
axs[0].imshow(img)
axs[0].set_title('Image de base')

axs[1].imshow(img_NB, cmap=plt.cm.gray)
axs[1].set_title('Image en N&B')

fig.tight_layout()

In [ ]:
# Teinture d'images en N&B : comment changer la valeur des pixels en donnant un facteur à chaque couche, pour "teinter" l'image
titre, img = chooseRandomImg(images)

# On commence par passer l'image en niveaux de gris
img_NB = skimage.color.rgb2gray(img)

# Puis on repasse le résultat en RGB
# Permet d'avoir une image visuellement noir et blanc mais avec trois couches
img_Tint = skimage.color.gray2rgb(img_NB)

# on définit un multiplicateur, sous la forme [x,y,z], ou chaque chiffre est un facteur qui sera appliqué à la valeur des pixels de la couche correspondante de l'image
r_multiplier = [1,0,0] # Par exemple, ici, on multipliera uniquement la couche rouge
g_multiplier = [0,1,0] # Et ici la couche verte
random_multiplier = [random.randint(1,100)/100,random.randint(1,100)/100, random.randint(1,100)/100] # On peut aussi donner des facteurs indépendants pour chaque couche, et aléatoire

img_Tint *= random_multiplier # On multiplie ensuite l'image par le multiplicateur

# Et on affiche les résultats
fig, axs = plt.subplots(ncols=3, nrows=1,figsize=(20,20))
axs[0].imshow(img)
axs[0].set_title('Image de base')
axs[0].set_axis_off()

axs[1].imshow(img_NB, cmap=plt.cm.gray)
axs[1].set_title('Image en N&B')
axs[1].set_axis_off()

axs[2].imshow(img_Tint)
axs[2].set_title('Image Teintée')
axs[2].set_axis_off()

fig.tight_layout()

In [ ]:
# Affichage des histogrammes d'une image
titre, img = chooseRandomImg(images)

# On va utiliser pour cela la fonction afficherImgHist qu'on a déclaré plus haut
# En lui donnant comme argument bins=256, ce qui correspond aux valeurs de pixels allant de 0 à 255 sur lesquelles on souhaite travailler
fig, axs = plt.subplots(ncols=1, nrows=2, figsize=(8,8))

afficherImgHist(img, axs, 256, titre=titre)

fig.tight_layout()

In [ ]:
# Normalisation d'histogramme avec image de référence
# Permet d'égaliser le contraste et la luminosité de deux images, en appliquant l'histogramme (en particulier cumulatif),
# de l'image de référence sur l'image de base

titreBase, imgBase = chooseRandomImg(images)
titreRef, imgRef = chooseRandomImg(images)

# On normalise deux images avec la fonction exposure.match_histogram(image de base, image référence, channel_axis=-1 (si l'image est en RGB))
imgNormalisee = skimage.exposure.match_histograms(imgBase, imgRef, channel_axis=-1)

# Pour chaque image, on va afficher l'image et ses histogrammes grâce à notre fonction afficherImgHist

# Pour cela on commence par créer une figure et ses axes avec la fonction subplots
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(8, 8))

# On transmets l'image de base et les axes de la première colonne à notre fonction
afficherImgHist(imgBase, axs[:, 0], titre=titreBase) # Le format axs[:, 0] permet de transmettre tous les axes de coordonnée x=0, quelque soit leur coordonnées y

# De même pour l'iamge de référence
afficherImgHist(imgRef, axs[:, 1], titre=titreRef)

# Et l'image normalisée
afficherImgHist(imgNormalisee, axs[:, 2], titre='Image normalisée') # On peut aussi donner un titre personnalisé à notre fonction

fig.tight_layout()

In [ ]:
# Egalisation de l'histogramme d'une image
# On peut aussi égaliser l'histogramme d'une image sans image de référence extérieure, pour s'assurer d'éviter une image trop sur ou sousexposée
titre, img = chooseRandomImg(images)

# Egalisation par étirement du contraste (contrast stretching) :
# On va récupérer les valeurs des pixels étant à 2 et à 98% de la plage dynamique actuelle de l'image,
# et étirer son histogramme pour que ces valeurs se retrouvent quasiment à 0 et à 255, en interpolant l'entre-deux.
p2, p98 = np.percentile(img, (2,98)) # On peut aussi travailler avec d'autres pourcentages, si souhaité

# exposure.rescale_intensity va effectuer le contrast stretching, elle prend comme argument l'image et le range avec lequel travailler
imgStretch = skimage.exposure.rescale_intensity(img, in_range=(p2,p98)) 

# Egalisation automatique :
# Va automatiquement lisser l'histogramme de l'image autour d'une valeur moyenne, relativement élevée
imgAutoEq = skimage.exposure.equalize_hist(img) # La fonction exposure.equalize_hist prend uniquement l'image en argument

# Egalisation adaptative :
# Lisse les valeurs de l'histogramme par plages de valeurs, en autorisant un écart maximal entre chacunes de ces plages
imgAdapt = skimage.exposure.equalize_adapthist(img, clip_limit=0.03) # exposure.equalize_adapthist prend l'image en argument, ainsi que la limite d'écart entre les plages

# On affiche les résultats comme précédemment :
fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(12,8))

# Image de base
afficherImgHist(img, axs[:, 0], titre='Image de base')

# Contrast stretching
afficherImgHist(imgStretch, axs[:, 1], titre='Contrast stretching')

# Auto-égalisation
afficherImgHist(imgAutoEq, axs[:, 2], titre='Egalisation automatique')

# Egalisation adaptative
afficherImgHist(imgAdapt, axs[:, 3], titre='Egalisation adaptative')

fig.tight_layout()

In [ ]:
# Egalisation locale d'une image
# L'égalisation locale d'une image permet d'égaliser les valeurs de pixels par zone d'une image.
# Une matrice de convolution à la distribution gaussienne et à la dimension spécifiée va lisser les valeurs de pixels de l'image entre elles par application localisées
# A NOTER : cette fonction fonctionne uniquement sur des images en N&B
titre, img = chooseRandomImg(images)
imgNB = skimage.color.rgb2gray(img)

# Pour l'égalisation locale :
# On va commencer par créer la matrice de convolution, appelée footprint
footprint = skimage.morphology.disk(30) # On crée une matrice avec distribution gaussienne (formant un disque de coefficients), de la taille choisie (ici 30)
imgLocEq = skimage.filters.rank.equalize(imgNB, footprint=footprint) # On utilise en suite la fonction filters.rank.equalize(image, footprint)

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12,8))

afficherImgHist(img, axs[:, 0], titre='Image de base')
afficherImgHist(imgNB, axs[:, 1], titre='Image N&B')
afficherImgHist(imgLocEq, axs[:, 2], titre='Image égalisée')

fig.tight_layout()
fig.show()

In [ ]:
# Passage en HSV et seuillage
# Le HSV (pour Hue (teinte), Saturation, Value (luminosité)) est un espace colorimétrique à trois couches, comme le RGB
# Il ne permet pas l'affichage photoréaliste des images, mais sert à stocker sur chaque couche des informations pour chaque pixels, 
# obtenues par écrasement pondéré des valeurs des trois couches RGB

titre, img = chooseRandomImg(images)
hsv_img = skimage.color.rgb2hsv(img) # On convertit l'image en HSV

# On va ensuite décomposer l'image comme suit, pour créer trois arrays qui contiennent chacun une des couches de l'image HSV
hue_img = hsv_img[:, :, 0] # La première couche correspond aux informations de teinte de chaque pixel
sat_img = hsv_img[:, :, 1] # La deuxième aux informations de saturation de chaque pixel
val_img = hsv_img[:, :, 2] # La troisième aux informations de luminosité

# Le passage en HSV peut être très utile pour "seuiller" une image, c'est-à-dire en faire une image binaire
# On va, par exemple, ne conserver que les pixels ayant une valeur de teinte ou de luminosité supérieure ou inférieure à un seuil fixé
# Cela va permettre d'obtenir une rapide et approximative détection des contours dans une image, notamment.
hue_threshold = 0.04
val_threshold = 0.02

# On seuille une image en récupérant un tableau de booléens, valant 0 ou 1 selon qu'ils passent la condition fixée
bin_img = (hue_img > hue_threshold) | (val_img < val_threshold)

# On affiche les résultats, sans histogramme cette fois
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(16,8))

axs[0,0].imshow(img)
axs[0,0].set_title('Image de base')
axs[0,0].set_axis_off()

axs[0,1].imshow(hsv_img)
axs[0,1].set_title('Image HSV')
axs[0,1].set_axis_off()

axs[0,2].imshow(hue_img, cmap='hsv')
axs[0,2].set_title('Couche de teinte')
axs[0,2].set_axis_off()

axs[1,0].imshow(sat_img, cmap='hsv')
axs[1,0].set_title('Couche de saturation')
axs[1,0].set_axis_off()

axs[1,1].imshow(val_img, cmap='hsv')
axs[1,1].set_title('Couche de luminosité')
axs[1,1].set_axis_off()

axs[1,2].imshow(bin_img, cmap=plt.cm.gray)
axs[1,2].set_title('Image seuillée')
axs[1,2].set_axis_off()

fig.tight_layout()

In [ ]:
# Ajouter du bruit dans une image
titre, img = chooseRandomImg(images)

# On va ajouter du bruit à notre image de base grâce à la fonction util.random_noise
# De nombreuses options produisant des types différents de bruit existent, on va ici en préseenter deux :

# Bruit Gaussien : le mode gaussien est le mode par défaut, sa précision est facultative. 
noise_img = skimage.util.random_noise(img, mode='gaussian', var=0.2, mean=0.4)
# var correspond à la variance de la distribution aléatoire. mean à la moyenne de cette même distribution

# Salt & Pepper : remplace des pixels aléatoirement sur chaque couche par 0 ou 1.
noise_img2 = skimage.util.random_noise(img, mode='s&p', amount=0.8) # amount définit la fréquence de remplacement des pixels

fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize=(12,12))
for ax in axs.flat:
    ax.set_axis_off()

axs[0].imshow(img)
axs[1].imshow(noise_img)
axs[2].imshow(noise_img2)

fig.tight_layout()

In [ ]:
# Débruitage d'une image - 1
# Ces différents algorithmes sont utilisés pour enlever le bruit d'une image, à partir d'une estimation de la déviation moyenne des pixels dans l'image
titre, img = chooseRandomImg(images)

# Bruitage de l'image, d'après une valeur sigma correspondant à la variation souhaitée
sigma = 0.2
noisy = skimage.util.random_noise(img, var = sigma**2)

# Total Variation : Débruite l'image d'après la moyenne de la variation des pixels sur l'image, pour lisser les zones où elle est supérieure à cette moyenne
# Le paramètre weight régule la "tolérance" de l'algorithme
imgTV = skimage.restoration.denoise_tv_chambolle(noisy, weight=0.4, channel_axis=-1)

# Bilatéral : Débruite chaque pixel en lissant sa valeur par rapport à la valeur des médiane des pixels environnants sur les deux directions
# Les paramètres sigma_color et sigma_spatial régule la tolérance de l'algo sur les variations de ces deux paramètres
imgBilat = skimage.restoration.denoise_bilateral(noisy, channel_axis=-1, sigma_color=0.02, sigma_spatial=5)

# Wavelet : Débruite l'image par lissage des valeurs de pixels selon une matrice de convolution concentrique
imgWavelet = skimage.restoration.denoise_wavelet(noisy, channel_axis=-1, rescale_sigma=True)

fig, axs = plt.subplots(nrows = 2, ncols = 2, figsize=(20,20))
for ax in axs.flat:
    ax.set_axis_off()

axs[0,0].imshow(noisy)
axs[0,0].set_title('Image bruitée')

#Débruitage par Total Variation
axs[0,1].imshow(imgTV)
axs[0,1].set_title('Total Variation')

#Débruitage par bilatéral
axs[1,0].imshow(imgBilat)
axs[1,0].set_title('Bilateral')

#Débruitage par wavelet
axs[1,1].imshow(imgWavelet)
axs[1,1].set_title('Wavelet')

fig.tight_layout()

In [ ]:
# Débruitage d'une image - 2 : Moyennant non-local
# Cet algorithme permet de débruiter une image par noyaux de convolution 
# en appliquant des opérations de moyennes sur les valeurs de pixels
# Il est plus efficace que ceux vu précédemment, mais souvent plus long à s'éxécuter
titre, img = chooseRandomImg(images)

# Bruitage de l'image
sigma = 0.4
noisy = skimage.util.random_noise(img, var=sigma**2)

# Estimation de la déviation moyenne standard de l'image,
# au cas où l'on a pas nous-même fixé la valeur sigma précédente. C'est-à-dire toujours, en conditions réelles
dev_estim = np.mean(skimage.restoration.estimate_sigma(noisy, channel_axis=-1))

# On prépare une variable de paramètres qu'on va adresser à l'algorithme
patch_kw = dict(patch_size=5, patch_distance=6, channel_axis=-1) 

# On utilise la fonction .restoration.denoise_nl_means
# Ses arguments minimums sont : l'image à traiter et le paramètre h=la tolérance aux déviations mesurées
denoise = skimage.restoration.denoise_nl_means(noisy, h=1.15 * dev_estim, fast_mode=False, **patch_kw)

# Le même algorithme, cette fois avec sigma fourni.
# Cela signifie qu'on lui transmet directement notre estimation de la deviation moyenne standard de l'image
denoise2 = skimage.restoration.denoise_nl_means(noisy, h=0.8 * dev_estim, sigma=dev_estim, fast_mode=False, **patch_kw)

# Sans sigma, mais avec le fast_mode activé
denoise_fast = skimage.restoration.denoise_nl_means(noisy, h=0.8 * dev_estim, fast_mode=True, **patch_kw)

# Avec sigma fourni et fast_mode activé
denoise2_fast = skimage.restoration.denoise_nl_means(noisy, h=0.8 * dev_estim, sigma=dev_estim, fast_mode=True, **patch_kw)

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,20))
for ax in axs.flat:
    ax.set_axis_off()

axs[0,0].imshow(noisy)
axs[0,0].set_title('image bruitée')

axs[0,1].imshow(denoise)
axs[0,1].set_title('algorithme lent')

axs[0,2].imshow(denoise2)
axs[0,2].set_title('algorithme lent avec sigma fourni')

axs[1,0].imshow(denoise_fast)
axs[1,0].set_title('algorithme rapide')

axs[1,1].imshow(denoise2_fast)
axs[1,1].set_title('algorithme rapide avec sigma fourni')

axs[1,2].imshow(img)
axs[1,2].set_title('originale')

fig.tight_layout()

In [ ]:
# Réparation par inpainting
# Sert à "réparer" une image en interpolant la valeur des pixels à partir des pixels environnants
titre, img = chooseRandomImg(images)

# Création d'un masque de "défauts" pour l'image
# On commencer par créer un tableau aux mêmes dimensions que notre image, d'une seule couche,
# pouvant contenir des booléens (sous la forme 0 ou 1) mais pour l'instant rempli de zéros
mask = np.zeros(img.shape[:-1], dtype=bool)

# Ajout de points de défauts aléatoires
rstate = np.random.default_rng(0)
for radius in [2, 4, 8, 64, 128]: # Le radius va déterminer la taille des défauts
    # Nous laissons moins de chances d'avoir des gros défauts
    thresh = 3+0.25*radius
    tmp_mask = rstate.standard_normal(img.shape[:-1]) > thresh
    if radius > 0:
        tmp_mask = skimage.morphology.binary_dilation(tmp_mask, skimage.morphology.disk(radius, dtype=bool))
    mask[tmp_mask] = 1

# On applique les défauts du masque au même endroit sur toutes les couches de l'image
# Pour ce faire, on multiplie les valeurs de chaque pixel sur chaque couche par l'inverse de la valeur de la même cellule dans le masque
# Ainsi, les cellules où l'on a mis un 1 multiplieront les valeurs du pixel par 0, créant une zone noire
img_defect = img * ~mask[..., np.newaxis]

# Puis on peut appeler la fonction .restoration.inpaint_biharmonic pour l'inpainting
# A noter, on doit transmettre à cette fonction un masque. Ici les défauts ont été créé par nous, donc aucun problème
# Dans le cas d'une image abimée de base, il peut être utile de d'abord recourir à une opération de seuillage, pour créer ce masque
img_result = skimage.restoration.inpaint_biharmonic(img_defect, mask, channel_axis=-1)

fig, axes = plt.subplots(ncols = 2, nrows = 2, figsize=(12,12))
for a in axs.flat:
    a.set_axis_off()

ax = axes.ravel() # La fonction ravel permet de réorganiser les axes sous la forme [0, 1, 2, 3, ...] 
# même quand ils seront affichés dans une grille à deux dimensions

ax[0].imshow(img)
ax[0].set_title('Image de base')

ax[1].imshow(mask, cmap=plt.cm.gray)
ax[1].set_title('Masque')

ax[2].imshow(img_defect)
ax[2].set_title('Image avec défaut')

ax[3].imshow(img_result)
ax[3].set_title('Image inpaintée')

fig.tight_layout()
fig.show()